# Projeto 2 - Visualização computacional

### João Paulo van Tol Amaral Guerra - Nusp 12557783
### Pedro Henrique Freitas Maiorano - Nusp 12701748

# Bibliotecas necessárias

## Pip Install para o colab

In [5]:
!pip install jupyter-dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.9 MB 40.8 MB/s 
     |████████████████████████████████| 1.6 MB 46.4 MB/s 


In [6]:
!pip install dash_bootstrap_components 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 26.6 MB/s 


## Bibliotecas

In [7]:
from jupyter_dash import JupyterDash
from dash import dcc,html
from dash.dependencies import Input, Output
import plotly.express as px
from dash import Dash, dash_table
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Dados e preparações

## Dados

In [8]:
data = pd.read_csv("https://raw.githubusercontent.com/PedroMaiorano/IMDB-Movies-Dataset/main/data.csv",index_col=0)

data_px= data.copy()

df_top = pd.read_csv("https://raw.githubusercontent.com/joaovantol/projeto1viscomp/main/df_top2.csv", header=(0)) #Base de dados  processada

df_top2 = df_top.copy()


## Preparações

In [9]:
L=data.Released_Year
Anos=[]
for i in L:
  if i not in Anos:
    Anos.append(i)
AnosCres=sorted(Anos)
AnosDecr=sorted(Anos,reverse=True)
max(Anos)


2020

In [10]:
df_top2['filmes']=df_top2['filmes'].map(str)
df_top2['ano_ultimo_filme']=df_top2['ano_ultimo_filme'].map(str)
df_top2['duracao_ultimo_filme']=df_top2['duracao_ultimo_filme'].map(str)

In [11]:
data_table=data.filter(['Series_Title','Released_Year','Genre','Director', 'Star1', 'Star2',
       'Star3', 'Star4', 'No_of_Votes'])

In [12]:
for i in range(df_top.shape[0]):
  if df_top['ano_ultimo_filme'][i] < 1981:
    df_top.loc[i,'grupo'] = '1920-1980'
  elif df_top['ano_ultimo_filme'][i] < 2001:
    df_top.loc[i,'grupo'] = '1981-2000'
  elif df_top['ano_ultimo_filme'][i] < 2011:
    df_top.loc[i,'grupo'] = '2001-2010'
  else:
    df_top.loc[i,'grupo'] = '2011-2020'

In [13]:
for i in range(data_px.shape[0]):
  if data_px['Released_Year'][i] < 1981:
    data_px.loc[i,'grupo'] = 0
  elif data_px['Released_Year'][i] < 2001:
    data_px.loc[i,'grupo'] = 1
  elif data_px['Released_Year'][i] < 2011:
    data_px.loc[i,'grupo'] = 2
  else:
    data_px.loc[i,'grupo'] = 3

In [14]:
def Frente_1(data_1,lista=[]):
  data_1_genero = data_1['Genre']
  data_1_genero=data_1_genero.to_numpy()
  Generos=[]
  NovaLista=[]
  dict={}

  for i in range(len(data_1)):
   k=data_1_genero[i].split(", ")
   NovaLista.append(k)

  for i in NovaLista:
     for j in i:
      if not j in Generos:
        Generos.append(j)
      dict[j] = dict.get(j, 0) + 1

  df1=pd.DataFrame(dict,index=[0])
  df1=df1.T
  df1=df1.reset_index()
  df1=df1.rename(columns={'index':'Gênero',0:'Quantidade'})
  df1=df1.sort_values(by='Quantidade')


  df2=pd.DataFrame(
    {
        "Gênero": [],
        "Quantidade": [],
    })




  df1=df1.sort_values("Quantidade", ascending=False)
  df1=df1.reset_index()
  soma=df1['Quantidade'].sum()
  Ant=0

  if len(lista) > 0:
    for i in (lista):
      K = df1["Gênero"] == i
      df2=pd.concat([df2,df1[K]])
  else:
    df2=df1

  df2=df2.sort_values("Quantidade", ascending=False)

  df2['Porcentagem']=  (df2['Quantidade'] / df2['Quantidade'].sum())

  df2=df2.reset_index()

  for i in range(1,len(df2)):
    Atual= df2['Porcentagem'].iloc[i]
    df2['Porcentagem'].iloc[i]=df2['Porcentagem'].iloc[i-1] + Atual



  return df2
 
Genero_Lista = Frente_1(data)




/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


# Dashboard

In [18]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.DARKLY])

app.title='Projeto 2 - Visualizaço Computacional'


app.layout =  html.Div(children=[
    html.H1('Projeto 2 - Visualizaço Computacional'),
    html.H5('João Paulo van Tol Amaral Guerra - Nusp 12557783'),
    html.H5('Pedro Henrique Freitas Maiorano - Nusp 12701748',style={'align-items': 'center', 'justify-content': 'center'}),

    dbc.Accordion(
        [ 
            #Item 1 
            dbc.AccordionItem(
                [
                    html.H1("Olá,"),
                 
                    html.P("Esse dashboard interativo é o resultado de um trabalho prático da discplina de visualização computacional, o dashboard foi todo desenvolvido na linguagem python utilizando bibliotecas conhecidas como: Plotly, Pandas, Dash e alguns de seus componentes."
                    " Também é válido ressaltar que é o código permite o implemento do ngrok para o compartilhamento do dashboard, retirando a limitação de ser apenas uma visualização local"),
                 
                    html.P("Aqui você encontrará visualizações no que diz respeito de filmes e atores feitos a partir da base de dados do IMDB. A base de dados em questão é resultado de uma votação do público que contempla o top 1000 filmes mais votados e as suas informações."),
                    
                    dbc.Button("Link dos dados",
                               id="link-centered",
                               className="me-1",
                               href='https://www.kaggle.com/datasets/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows'),
                    
                    dbc.Button("Vídeo Apresentação",
                               id="link-apresentação",
                               className="me-1",
                               href='https://www.youtube.com/watch?v=DV2VG7ndiUM')
                ],
                title="Informações do dashboard",
            ),
         



            #Item 2
            dbc.AccordionItem(
                [
                    
                    #Coordenadas Paralelas
                    html.Div(children='Coordenadas Paralelas'),
                      dcc.Graph(id='Parallel'),
            


                  ##Slider intervalo do ranking
                    html.Div([dbc.Label("Ranking de filmes"),
                              dcc.RangeSlider(id="Intervalo do ranking",
                                              min=1,
                                              max=1000,
                                              value=[250,750],
                                              marks=None,
                                              step=1,
                                              tooltip={"placement": "bottom", "always_visible": True})
                              ]),
                 



                 ##Dropdown Genero do filme
                    html.Div([dbc.Label("Gênero do filme"),
                               dcc.Dropdown( id = 'Gênero',
                                  options=[{"label": f"{i}", "value": i} for i in Genero_Lista['Gênero'].to_list()],
                                  value = Genero_Lista['Gênero'].to_list(),
                                  multi = True,
                                  style={'color': 'Black', 'font-size': 20})
    ]),
                    



                 ##Slider intervalo do ano
                    html.Div([dbc.Label("Ano do Filme"),
                              dcc.RangeSlider(id="Intervalo do ano",
                                              min=1920,
                                              max=2020,
                                              value=[1920,2020],
                                              marks=None,
                                              step=1,
                                              tooltip={"placement": "bottom", "always_visible": True})
                              ]),
                 

                 ##Gráfico contagem de gêneros
                  html.Div(children='Diagrama de Pareto'),dcc.Graph(id='graph-countgen'),



                 #Tabela
                  html.Div(id='tabela')



                ],
                title="Visualizações dos filmes",
            ),
         




            #Item 3
            dbc.AccordionItem(
                [
                    #Gráfico
    html.Div(children='Scatter Plot'),dcc.Graph(id='graph-with-slider'),



    #Slider Número de atores                   
    html.Div([dbc.Label('Número de Atores'),
     dcc.Slider(
        id='Número-Atores',
        min=100,
        max=2273,
        marks=None,
        value=500,
        step=1,
        tooltip={"placement": "bottom", "always_visible": True})]),
    

    #Radio Escala Eixo X
    html.Div([ dbc.Label('Escala do Eixo X'),                  
    dbc.RadioItems(id='Escala-X',
                   options=[{"label": "Decimal", "value": "Decimal"},
                            {"label": "Logarítmico", "value": "Logaritmico"}],
                   value='Decimal',
                  inline=True)]),
                 
           
    #Dropdown Eixo X
    html.Div([dbc.Label('Eixo X'),
    dcc.Dropdown(id='Var-X', 
                 options=['bilheteria','número de votos'],
                style={'color': 'Black', 'font-size': 20},
                 value='bilheteria')]),


    #Dropdown Eixo Y
    html.Div([dbc.Label('Eixo Y'),           
    dcc.Dropdown(id='Var-Y', 
                 options=['nota do publico','nota dos críticos'],
                 style={'color': 'Black', 'font-size': 20},
                 value='nota do publico')]),  

    ##Dash Tabela
    html.Div(['Tabela',
    dash_table.DataTable(columns=[
        {"name": "Ator", "id": "ator"},
        {"name": "Nº de filmes", "id": "filmes"},
        {"name": "Último filme", "id": "ultimo_filme"},
        {"name": "Ano do último filme", "id": "ano_ultimo_filme"},
        {"name": "Duração do último filme", "id": "duracao_ultimo_filme"},
        {"name": "Gênero do último filme", "id": "genero_ultimo_filme"},
        {"name": "Diretor do último filme", "id": "diretor_ultimo_filme"}],
        data=df_top2.to_dict("records"),
        filter_action='native',
        page_size=20,
        style_data={
        'color': 'black',
        'backgroundColor': 'white',
        'whiteSpace': 'normal',
            'height': 'auto',
            'textAlign': 'left'},
        style_data_conditional=[
        {
            'if': {'row_index': 'odd'},
            'backgroundColor': 'rgb(220, 220, 220)',
        }],
        style_header={
        'backgroundColor': 'rgb(210, 210, 210)',
        'color': 'black',
        'fontWeight': 'bold'},
        
        editable=True)
        ])
        ],
        title="Visualizações dos atores",
            )],
            always_open=True
            
            
            ),
    
    
])




#CallBack Item2 - Coordenadas paralelas
@app.callback(
    Output('Parallel','figure'),
    Input("Intervalo do ranking",'value'),
    Input('Gênero','value'),
    Input("Intervalo do ano",'value'))

def update_figure(Intervalo_do_ranking,Genero,Intervalo_do_ano):
  filtered_data = data_px.copy()
  

  RankMin=Intervalo_do_ranking[0]
  RankMax=Intervalo_do_ranking[1]
  filtered_data=filtered_data[filtered_data.index >= RankMin]
  filtered_data=filtered_data[filtered_data.index <= RankMax]



  AnoMin=Intervalo_do_ano[0]
  AnoMax=Intervalo_do_ano[1]

  filtered_data=filtered_data[filtered_data["Released_Year"] >= AnoMin]
  filtered_data=filtered_data[filtered_data["Released_Year"] <= AnoMax]


  J=filtered_data.Released_Year
  Anos=[]
  for i in J:
    if i not in Anos:
     Anos.append(i)

  AnosCres=sorted(Anos)
  AnosDecr=sorted(Anos,reverse=True)

  filtered_data['Meta_score']=filtered_data['Meta_score'].fillna(0)

  data_gen=Frente_1(filtered_data,Genero)

  filtered_data['Genre']=filtered_data['Genre'].str.split(", ")

  
  filtered_data=filtered_data.reset_index()
  
  for i in range(len(filtered_data)):
    contagem=0
    for j in filtered_data['Genre'][i]:
      for m in data_gen['Gênero']:
        if j == m:
          contagem = contagem +1
    if contagem == 0:
      filtered_data=filtered_data.drop(i)

  
  fig = px.parallel_coordinates(filtered_data, color="grupo",
                                dimensions=['Released_Year', 'Runtime','IMDB_Rating','Meta_score','No_of_Votes'],
                                color_continuous_scale=px.colors.cyclical.mygbm,
                              range_color=[0,4],
                              color_continuous_midpoint=2,
                              labels={"grupo":"Grupo","Released_Year":"Ano de Lançamento", "Runtime":"Tempo de Duração", 'IMDB_Rating':"Nota IMDB",'Meta_score':"Nota Crítica", 'No_of_Votes':"Votos do Público"})
  
  

  return fig





#CallBack Item2 - Gráfico de gênero
@app.callback(
    Output('graph-countgen','figure'),
    Input("Intervalo do ranking",'value'),
    Input('Gênero','value'),
    Input("Intervalo do ano",'value')
)

def update_figure(Intervalo_do_ranking,Genero,Intervalo_do_ano):
  filtered_data = data.copy()

  RankMin=Intervalo_do_ranking[0]
  RankMax=Intervalo_do_ranking[1]
  filtered_data=filtered_data[filtered_data.index >= RankMin]
  filtered_data=filtered_data[filtered_data.index <= RankMax]



  AnoMin=Intervalo_do_ano[0]
  AnoMax=Intervalo_do_ano[1]

  filtered_data=filtered_data[filtered_data["Released_Year"] >= AnoMin]
  filtered_data=filtered_data[filtered_data["Released_Year"] <= AnoMax]

  data_gen=Frente_1(filtered_data,Genero)

  trace1 = go.Bar(
    x=data_gen['Gênero'],
    y=data_gen['Quantidade'],
    name='Gênero',
    yaxis='y',
    marker=dict(
        color='rgb(34,163,192)'))
  
  trace2 = go.Scatter(
      x=data_gen['Gênero'],
      y=data_gen['Porcentagem'],
      yaxis="y2",
      name='Porcentagem Acumulada',
      )



  fig = go.Figure()

  fig.add_trace(trace1)

  fig.add_trace(trace2)


  fig.update_layout(xaxis=dict(domain=[0, 1]),
            
    yaxis=dict(
        title="Quantidade de filmes",
        titlefont=dict(color="#1f77b4"),
        tickfont=dict(color="#1f77b4")),
                  
    #create 2nd y axis      
    yaxis2=dict(title="Porcentagem Acumulada",overlaying="y",
                side="right",position=1,range=[0,1]))


  
  
  return fig


#Callback item 2 - tabela
@app.callback(
    Output('tabela','children'),
    Input("Intervalo do ranking",'value'),
    Input('Gênero','value'),
    Input("Intervalo do ano",'value'))

def update_figure(Intervalo_do_ranking,Genero,Intervalo_do_ano):
    filtered_data = data_table.copy()
    filtered_data=filtered_data.sort_values(by='No_of_Votes', ascending= False)
    filtered_data = filtered_data.reset_index()
    filtered_data=filtered_data.drop(columns='index')
    filtered_data = filtered_data.reset_index()
    for i in range(len(filtered_data)):
      filtered_data['index'][i]=filtered_data['index'][i]+1


    RankMin=Intervalo_do_ranking[0]
    RankMax=Intervalo_do_ranking[1]

    filtered_data=filtered_data[filtered_data['index'] >= RankMin]
    filtered_data=filtered_data[filtered_data['index'] <= RankMax]



    AnoMin=Intervalo_do_ano[0]
    AnoMax=Intervalo_do_ano[1]

    filtered_data=filtered_data[filtered_data["Released_Year"] >= AnoMin]
    filtered_data=filtered_data[filtered_data["Released_Year"] <= AnoMax]


    data_gen=Frente_1(filtered_data,Genero)


    filtered_data=filtered_data.rename(columns={'Series_Title': "Nome do filme",'Released_Year':"Ano de lançamento",'Genre':"Gênero","Director":"Diretor", "Star1":"Estrela 1", "Star2":"Estrela2", "Star3":"Estrela 3", "Star4":"Estrela 4","No_of_Votes":"Votos do Público",'index': 'Posição no ranking'})
    filtered_data['Ano de lançamento']=filtered_data['Ano de lançamento'].map(str)
    filtered_data["Votos do Público"]=filtered_data["Votos do Público"].map(str)
    filtered_data['Posição no ranking']=filtered_data['Posição no ranking'].map(str)
   
   
   
    tab = dash_table.DataTable(
            id='table',
            columns=[{"name":i, "id": i} for i in filtered_data.columns],
            data=filtered_data.to_dict('records'),
            filter_action='native',
            page_size=20,
            style_data={
            'color': 'black',
            'backgroundColor': 'white',
            'textAlign': 'left',
            'whiteSpace': 'normal',
            'height': 'auto'},
            style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(220, 220, 220)',
            }],
            style_header={
            'backgroundColor': 'rgb(210, 210, 210)',
            'color': 'black',
            'fontWeight': 'bold',
            'textAlign': 'left'},
            editable=True)
    
    J= html.Div([tab])
    return J




#Callback Item 3
@app.callback(
  Output('graph-with-slider','figure'),
  Input('Número-Atores','value'),
  Input('Escala-X','value'),
  Input('Var-X','value'),
  Input('Var-Y','value'))

def update_figure(Número_Atores,Escala,Var_X,Var_Y):
  filtered_df = df_top.iloc[0:Número_Atores]

  if Escala == 'Decimal':
        escala = False
  else:
    escala = True
  
  if Var_X == 'bilheteria':
    X = 'bilheteria'
    
  else:
    X = 'votos'
  
  if Var_Y == 'nota do publico':
    Y = 'nota_publico'
  else:
    Y = 'nota_critica'


  fig = px.scatter(filtered_df, x=X, y=Y,
                   size='filmes', size_max=50,
                    range_x=[1,3000],range_y=[3, 10],
                    log_x=escala, # diminui a quantidade de espaço vazio
                    color='grupo',
                    color_discrete_sequence=px.colors.qualitative.Dark2,
                    hover_name='ator',
                    labels={
                        "x": str(Var_X),
                        "y": str(Var_Y),
                        "color": "Grupo",
                        "size": "Número de filmes"})

  fig.update_layout({"margin": {
        "t": 60, "b": 5, "l": 5, "r": 5},
    "legend": {
        "title": "",
        "orientation": "h", "yanchor": "bottom",
        "y": 1.02, "xanchor": "right", "x":1}})
  
  fig['layout']['xaxis'].update(autorange = True)

  return fig



app.run_server(debug=True)



Dash app running on:


<IPython.core.display.Javascript object>